In [1]:
from translate import *

df = pd.read_csv("01.csv")
df['translate_claude'] = df.apply(lambda x: "NA", axis=1)
for i in tqdm(range(len(df))):
    text = df['Transcript'].iloc[i]
    translated_text = get_translate(text)
    df.at[i, 'translate_claude'] = translated_text  
df.to_csv("01_translate_claude.csv", index=False, progress_bar=True)

  1%|          | 1/195 [00:02<07:59,  2.47s/it]